In [15]:
#Libraries
import numpy as np # 
import pandas as pd 

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
PATH= '/content/drive/My Drive/'

In [0]:
raw_dataset = pd.read_csv(PATH + 'Sentiment.csv')
dataset = raw_dataset.copy()

In [0]:
dataset = dataset[['text','sentiment']]

In [0]:
dataset['text'] = dataset['text'].apply(lambda x: x.lower())
dataset['text'] = dataset['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [20]:
print(dataset[ dataset['sentiment'] == 'Positive'].size)
print(dataset[ dataset['sentiment'] == 'Neutral'].size)
print(dataset[ dataset['sentiment'] == 'Negative'].size)

4472
6284
16986


In [0]:
for idx,row in dataset.iterrows():
    row[0] = row[0].replace('rt',' ')

In [22]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(dataset['text'].values)
X = tokenizer.texts_to_sequences(dataset['text'].values)
X = pad_sequences(X)
X

array([[   0,    0,    0, ...,   51, 1039,    1],
       [   0,    0,    0, ..., 1577, 1356,  847],
       [   0,    0,    0, ...,   10,  696,  518],
       ...,
       [   0,    0,    0, ...,   68,   62,    3],
       [   0,    0,    0, ..., 1112, 1588,   81],
       [   0,    0,    0, ...,  196,    3,  880]], dtype=int32)

In [0]:
len(dataset['text'].values)

13871

In [0]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 28, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
#Position 0 - negative
#Position 1 - neutral
#Position 2 - positive

Y = pd.get_dummies(dataset['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(9293, 28) (9293, 3)
(4578, 28) (4578, 3)


In [0]:
X_train


array([[   0,    0,    0, ...,  811,  161,  540],
       [   0,    0,    0, ...,   91,  539,    1],
       [   0,    0,    0, ..., 1992,   72, 1651],
       ...,
       [   0,    0,    0, ...,  976,   14,    1],
       [   0,    0,    0, ...,   37, 1478, 1424],
       [   0,    0,    0, ...,  223,   12,    1]], dtype=int32)

In [0]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/7





 - 24s - loss: 0.8492 - acc: 0.6327
Epoch 2/7
 - 14s - loss: 0.7096 - acc: 0.6962
Epoch 3/7
 - 14s - loss: 0.6485 - acc: 0.7253
Epoch 4/7
 - 14s - loss: 0.6052 - acc: 0.7488
Epoch 5/7
 - 14s - loss: 0.5763 - acc: 0.7612
Epoch 6/7
 - 14s - loss: 0.5420 - acc: 0.7848
Epoch 7/7
 - 14s - loss: 0.5185 - acc: 0.7862


In [0]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.80
acc: 0.68


## Testing




In [0]:
twt = ["The good thing is: The book is not restricted to just music. Music is just one use-case. Natural Language Processing for generating texts, GANs and Autoencoders for Images, you name it."]
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
sentiment = model.predict(twt,batch_size=1,verbose = 3)[0]
print(sentiment)







[0.33530623 0.3319023  0.3327915 ]


In [0]:
from sklearn.externals import joblib
joblib.dump(model,"sentiment_model.pkl")

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['sentiment_model.pkl']

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
model_file = drive.CreateFile({'title' : 'sentiment_model.pkl'})                       
model_file.SetContentFile('sentiment_model.pkl')                       
model_file.Upload()